# __Step 7.1a: Testing nominatim container__

## ___Note___

This has been a hell of a ride for three weeks. The main obstacles are:
- Familiarity with docker
- Familiarity with nominatim server, particularly on how to get it up and running with very large osm.pbf files.
- Familiarity with nominatim docker, particularly on how export/import containers, setup persistent container (so the database is mounted), export/important a persistent database.

What I did wrong:
- When I run the container, I did not use the persistent container option. 

## ___Approach going forward___

### Initial server run

- Use the FREEZE parameter
  - See [this](https://nominatim.org/release-docs/latest/admin/Advanced-Installations/): database looses the ability to receive further data updates but the resulting database is only about a third of the size of a full database
- Set up local bind mount to /nominatim/data
- Set up mount nominatim-data

In [ ]:
'''
docker run -it \
  -e PBF_PATH=/nominatim/data/north-america-latest.osm.pbf \
  -e FREEZE=true \
  -p 8080:8080 \
  -v nominatim-data-na:/var/lib/postgresql/14/main \
  -v /home/shius/data_nominatim/continent_osm/:/nominatim/data \
  --name nominatim_na \
  mediagis/nominatim:4.2
'''
# Start time: 2023-03-16 17:35:17 (container time)

### Container export/import

In [ ]:
'''
# Export container
cd /home/shius/projects/plant_sci_hist/7_countries/nominatim_containers
docker ps
docker export 95784ce8c0a6 > nominatim_north-america.tar

# Export volume


# Import container
cat nominatim_north-america.tar | docker import - nominatim_persistent:latest
docker images       # A new image loaded

# Import volume

'''

## ___Note on local Nominatim install___

Using the Docker version I could just specify the different continent in different runs. But that is very clunky. So I am going to try install locally. Um... just too many dependent packages... Not doing this for now, stop at ICU install.

Following the [instruction](https://nominatim.org/release-docs/latest/admin/Installation/). More specifically, [for Ubuntu 20.04](https://nominatim.org/release-docs/latest/appendix/Install-on-Ubuntu-20/)


### Installing the Required Software



In [ ]:
'''
sudo apt update -qq
sudo apt install -y php-cgi
sudo apt install -y build-essential cmake g++ libboost-dev libboost-system-dev \
                    libboost-filesystem-dev libexpat1-dev zlib1g-dev \
                    libbz2-dev libpq-dev liblua5.3-dev lua5.3 \
                    postgresql-12-postgis-3 \
                    postgresql-contrib-12 postgresql-12-postgis-3-scripts \
                    php-cli php-pgsql php-intl libicu-dev python3-dotenv \
                    python3-psycopg2 python3-psutil python3-jinja2 \
                    python3-icu python3-datrie python3-yaml
'''

### System Configuration

#### Creating Dedicated User Accounts

In [ ]:
'''
# Nominatim will run as a global service on your machine. It is therefore best 
# to install it under its own separate user account
sudo useradd -d /srv/nominatim -s /bin/bash -m nominatim

# export user name and home directory 
export USERNAME=nominatim
export USERHOME=/srv/nominatim

# Make sure that system servers can read from the home directory
# NOTE: add sudo otherwide `Operation not permitted`
sudo chmod a+x $USERHOME
'''

#### Setting up PostgreSQL

[Tuning the PostgreSQL database for nominatim](https://nominatim.org/release-docs/latest/admin/Installation/#tuning-the-postgresql-database)
- Use recommended values except for:
  - maintenance_work_mem = 5GB instead of 10GB

```
shared_buffers = 2GB
maintenance_work_mem = (10GB)
autovacuum_work_mem = 2GB
work_mem = (50MB)
effective_cache_size = (24GB)
synchronous_commit = off
max_wal_size = 1GB
checkpoint_timeout = 10min
checkpoint_completion_target = 0.9
autovacuum_max_workers = 1
fsync = off
full_page_writes = off
```

- Don't forget to re-enable them after the initial import:
```
fsync = on
full_page_writes = on
```

In [ ]:
'''
sudo vim /etc/postgresql/12/main/postgresql.conf
'''

In [ ]:
'''
# restart the postgresql service after updating this config file
sudo systemctl restart postgresql

# Get `System has not been booted with systemd as init system...`
# see: https://stackoverflow.com/questions/70332810/postgresql-problem-system-has-not-been-booted-with-systemd-as-init-system-pid

sudo service postgresql status
sudo service postgresql start

# Add two postgres users
# 1. user that does the import
# 2. another for the webserver
sudo -u postgres createuser -s $USERNAME
sudo -u postgres createuser www-data
'''

### Installing Nominatim

#### Building and Configuration

In [ ]:
'''
cd $USERHOME
wget https://nominatim.org/release/Nominatim-4.2.1.tar.bz2
tar xf Nominatim-4.2.1.tar.bz2
'''

## ___Additional testing___

### Try to get nominatim running after exporting

On 3/14/23, found issues with the nominatin runs where some address should be US but was pointed to China. Found this for records with "New Heaven" but the State name and postal code should be used before that anyway, so not sure what's going on. Beacuse of this, I need to rerun North America nominatim. I generated an export of container after the database was built but run into issues about getting it to run again. Here are the things I have tried:

- Import container
  - `docker import nominatim_north-america.tar mediagis/nominatim:4.2`
- Just do a straight call like how I build the databse
  - Does not work, ask for command
- Add `/app/start.sh` as command
  - Does not work:
    - `sed: can't read /.env: No such file or directory`
    - `chown: missing operand after 'nominatim:nominatim`
- Try [this solution](https://github.com/mediagis/nominatim-docker/discussions/306)
- Also see [this](https://github.com/mediagis/nominatim-docker/discussions/280)
- And [this](https://github.com/mediagis/nominatim-docker/discussions/351)

Try persistent container
- [Nominatim instruction](https://github.com/mediagis/nominatim-docker/discussions/351)
- [Backup, restore, migrate data volumes](https://docs.docker.com/storage/volumes/#back-up-restore-or-migrate-data-volumes)

#### Run, export, stop, remove, then import

In the following, I :
- Run nominatim docker, download Andorra osm.pbf data, build database using the persistent container setting.
- Export the container with a docker volume setup
- Stop and remove the container, delete the loaded image.
- Dockerd is still running so the docker volume is still there.
  - The image created (nominatim_persistent:latest) is also still there.
- Import the exported container
- Run the imported image by passing proper variables.
  - Specifically: -e PROJ_DIR and -v

In [ ]:

'''docker run -it --shm-size=1g \
  -e PBF_URL=https://download.geofabrik.de/europe/andorra-latest.osm.pbf \
  -e FREEZE=true \
  -v nominatim-data:/var/lib/postgresql/14/main \
  -p 8080:8080 \
  --name nominatim_andorra \
  mediagis/nominatim:4.2'''

# Export/import/run
'''
docker ps
docker volume ls    # Check volume exist
docker export -o test_persistent_andorra.tar 23d68084ca58

docker rm -f 23d68084ca58

docker ps -a        # check container is no longer there

# Load container
cat test_persistent_andorra.tar | docker import - nominatim_persistent:latest
docker images       # A new image loaded
'''

##### Solution 1: enter container and edit /app/start.sh

For a while, I am stuck when do docker run with `/app/start.sh` where the following error occurs:
- `chown: missing operand after 'nominatim:nominatim'`
- Looking into the shell script, the `chomn` commnad is followed by an environmental variable `${PROJECT_DIR}` which is not set.
- The following finally worked.

In [ ]:
# Solution 1
# Enter container and edit /app/start.sh
'''
# Run container
# Use /app/start.sh failed.
docker run -it \
  -e PBF_URL=https://download.geofabrik.de/europe/andorra-latest.osm.pbf \
  -e FREEZE=true \
  -p 8080:8080 \
  -v nominatim-data:/var/lib/postgresql/14/main \
  --name nominatim_rec \
  nominatim_persistent:latest \
  /bin/bash
'''

# Enter container
'''
apt-get update
apt-get install vim
vim /app/start.sh

# In VIM
# Add a line before IMPORT_FINISHED
PROJECT_DIR=/nominatim

# Back in bash
/app/start.sh
'''

##### Solution 2: pass the environmental variable

This works also, much easier. Finally.

In [ ]:
# Enter container and edit /app/start.sh
'''
docker ps -a     # make sure the previous container has stopped
docker images    # confirm nominatim_persistent:latest exist
docker volume ls # confirm nominatim-data exist

docker run -it \
  -e PBF_URL=https://download.geofabrik.de/europe/andorra-latest.osm.pbf \
  -e PROJECT_DIR=/nominatim \
  -e FREEZE=true \
  -p 8080:8080 \
  -v nominatim-data:/var/lib/postgresql/14/main \
  --name nominatim_rec \
  nominatim_persistent:latest \
  /app/start.sh
'''

#### Retry the above using a local osm.pbf file

To see if the following works:
- Set PBF_FILE environmental variable
- Set a bind mount for the osm.pbf data folder

All the above still works

In [ ]:
# Run container and create data volume: Ok
'''
docker run -it  \
  -e PBF_PATH=/nominatim/data/andorra-latest.osm.pbf \
  -e FREEZE=true \
  -v nominatim-data-test:/var/lib/postgresql/14/main \
  -v /home/shius/projects/plant_sci_hist/7_countries/testing:/nominatim/data \
  -p 8080:8080 \
  --name nominatim_andorra \
  mediagis/nominatim:4.2

docker ps
docker volume ls    # Check volume exist
'''

# Export/import/run
'''
docker export -o test_persistent_andorra2.tar d6f6a0e256e6

docker rm -f 23d68084ca58

docker ps -a        # check container is no longer there

# Load container
cat test_persistent_andorra.tar | docker import - nominatim_persistent:latest
docker images       # A new image loaded
'''

#### Use an existing volume 

##### By starting an unrelated (i.e., different location) nominatim container

The naminatim-data volume is created using Andorra data, want to see if I start a container specifying a different location but using the Andorra volume, what happens:
- Delete images
- Run nominatim docker
  - Specify PBF_URL to download equatorial-guinea
  - Specify location of equatorial-guinea osm.pbf
  - Specify nominatim-data mount 
- Testing
  - Search Andorra returned value.
  - Search Malabo (Eq Guinea's captial) returned nothing
  - So works.

In [ ]:
'''
# Remove image, this does not really matter since I am not using that docker
# container. Just to make this cleaner
docker images
docker image remove -f nominatim_persistent

docker run -it \
  -e PBF_PATH=PBF_PATH=/nominatim/data/equatorial-guinea-latest.osm.pbf \
  -e FREEZE=true \
  -p 8080:8080 \
  -v nominatim-data:/var/lib/postgresql/14/main \
  --name nominatim_eg \
  mediagis/nominatim:4.2
'''

##### By using an imported data volume

Next I want to see if I can restore the data volume after exporting.
- See [this section](https://docs.docker.com/storage/volumes/#back-up-restore-or-migrate-data-volumes) on backup
- This seems complocated.

In [ ]:
# Run persistent container, building database with a local file
# -p setting doc:
# https://www.mend.io/free-developer-tools/blog/docker-expose-port/
'''
docker run -it \
  -e PBF_PATH=PBF_PATH=/nominatim/data/andorra-latest.osm.pbf \
  -e FREEZE=true \
  -p 2346:2346 \
  -v nominatim-data-test:/var/lib/postgresql/14/main \
  -v /home/shius/projects/plant_sci_hist/7_countries/testing:/nominatim/data \
  --name nominatim-test \
  mediagis/nominatim:4.2
'''

### Initial nominatim setup note

#### [mediagis/nominatim-docker](https://github.com/mediagis/nominatim-docker/blob/master/4.2/README.md)

Starting out with this, realize that different regions need to be downloaded.

In [ ]:
'''
TESTING
---
# Start Nominatim container in another terminal
docker run -it \
  -e PBF_URL=https://download.geofabrik.de/europe/monaco-latest.osm.pbf \
  -e REPLICATION_URL=https://download.geofabrik.de/europe/monaco-updates/ \
  -p 8080:8080 \
  --name nominatim \
  mediagis/nominatim:4.2

# Find container id in another terminal, for this instance: cce4a525a8e2
docker ps

# Open another terminal and copy planet osm psf file into container
# Actually, this is not necessary, because I can use bind mounts (docker run -v)
#docker cp ~/data_nominatim/planet-230213.osm.pbf cce4a525a8e2:/nominatim/

# Start bash shell for the container and check if the file is there
docker exec -it cce4a525a8e2 bash
ls /nominatim/ # <--- confirmed

# Commit the container
docker commit -m "With planet data" -a "Shinhan Shiu" cce4a525a8e2 shius/nominatim_planet
docker images

# Save image
docker save shius/nominatim_planet > nominatim_planet.tar

# Stop and remove docker container
docker stop cce4a525a8e2
docker rm cce4a525a8e2

# Run this again to test local .pbf
docker run -it \
  -e PBF_PATH=/nominatim/data/monaco-latest.osm.pbf \
  -p 8080:8080  \
  -v /osm-maps/data:/nominatim/data \
  --name nominatim \
  mediagis/nominatim:4.2

docker run -it \
  -e PBF_PATH=/nominatim/planet-230213.osm.pbf \
  -p 8080:8080  \
  -v /osm-maps/data:/nominatim/ \
  --name nominatim_planet \
  shius/nominatim_planet
'''

In [ ]:
# Try michigan
'''bash
docker run -it \
  -e PBF_URL=https://download.geofabrik.de/north-america/us/michigan-latest.osm.pbf \
  -p 8080:8080  \
  --name nominatim_mi \
  mediagis/nominatim:4.2
'''


#### [Aximem/nominatim-docker-multiple-regions](https://github.com/Aximem/nominatim-docker-multiple-regions)

Not well documented and it did not work for me.

In [ ]:
'''bash
# In /home/shius/github
git clone https://github.com/Aximem/nominatim-docker-multiple-regions.git
cd nominatim-docker-multiple-regions/

# Build image
docker build --pull --rm -t nominatim .

# update file: multiple_regions/init_multiple_regions.sh
# Issue: Not sure where to find this file.
#COUNTRIES="africa australia-oceania europe antarctica central-america south-america asia"

# init multiple regions: did not work...
docker run -t -v /home/shius/data_nominatim/:/data nominatim sh /app/multiple_regions/init.sh
'''

### The following does not work

#### Set up test donor and recipient nominatins

In [ ]:
# Get osm.pbf
'''
wget https://download.geofabrik.de/africa/equatorial-guinea-latest.osm.pbf
wget https://download.geofabrik.de/europe/andorra-latest.osm.pbf
'''

# Start recipient server
'''
docker run -it \
  -e PBF_PATH=/nominatim/data/andorra-latest.osm.pbf \
  -e FREEZE=true \
  -p 8080:8080 \
  -v /home/shius/projects/plant_sci_hist/7_countries/testing:/nominatim/data \
  --name nominatim_rec \
  mediagis/nominatim:4.2
'''

# Export/import/run
'''
docker export -o test_andorra.tar 47ad114454aa

docker stop 47ad114454aa
docker rm 47ad114454aa

cat test_andorra.tar | docker import - nominatim_andorra:latest

docker run -it \
  -e PBF_PATH=/nominatim/data/andorra-latest.osm.pbf \
  -e FREEZE=true \
  -p 8080:8080 \
  -v nominatim-data:/var/lib/postgresql/14/main \
  --name nominatim_rec \
  nominatim_andorra:latest \
  /app/start.sh
'''
# The above doe not work


In [ ]:
# Start donor server
'''
docker run -it \
  -e PBF_PATH=PBF_PATH=/nominatim/data/equatorial-guinea-latest.osm.pbf \
  -e FREEZE=true \
  -p 808:8080 \
  -v ~/projects/plant_sci_hist/7_countries/testing:/nominatim/data \
  -v nominatim-data:/var/lib/postgresql/14/main \
  --name nominatim_don \
  mediagis/nominatim:4.2

docker ps

# Go to recipient container
docker exec -it e01ff686f768 /bin/bash
# In recipient container
cd /var/lib/postgresql/14
du main                                   # 327096  main
exit

# Go to doner container
docker exec -it 35c4c0cdfb93 /bin/bash
cd /var/lib/postgresql/14
du main                                   # 397388  main
exit
'''

#### Try 1: Copy database main folder over

In [ ]:
'''
# In host, copy donor postgresql database
docker cp 35c4c0cdfb93:/var/lib/postgresql/14/main ./  # copy files
du main/                                  # 397044  main/

# Test copy
mkdir -p test_dir/main
cp -r main/ test_dir/main/

# Replace recipient postgresql database
docker ps                                 # container id: e01ff686f768
docker cp main/ e01ff686f768:/var/lib/postgresql/14/

# Go to recipient container to change ownership
docker exec -it e01ff686f768 /bin/bash
chown -R postgres:postgres /var/lib/postgresql/14/main

# Test http://localhost:8080/search.php?q=usa
code: 500
Message: 'Failed to establish database connection:SQLSTATE[08006]

# Stop the server than restart
docker stop e01ff686f768
docker start e01ff686f768

# Cannot restart
'''

#### Try 2: Use a read-only bind mount

See read-only bind mount secton of [this](https://docs.docker.com/storage/bind-mounts/)
- This failed: `FATAL: Creating new database failed`

In [ ]:
# In ~/projects/plant_sci_hist/7_countries/testing
'''
docker run -it \
  -e PBF_URL=https://download.geofabrik.de/europe/andorra-latest.osm.pbf \
  -p 8080:8080 \
  --name nominatim_rec \
  -v main:/var/lib/postgresql/14/main:ro \
  mediagis/nominatim:4.2
'''

#### Try 3: Use a bind mout that's writable

Rid of the `ro` parameter
- Does not work.
- The resulted database is overwritten.
- Or the mount has never worked because the wsl copy remain the same, just the folder in the container is not what it is supposed to be.

In [ ]:
# In ~/projects/plant_sci_hist/7_countries/testing
'''
docker run -it \
  -e PBF_URL=https://download.geofabrik.de/europe/andorra-latest.osm.pbf \
  -p 8080:8080 \
  --name nominatim_rec \
  -v main:/var/lib/postgresql/14/main \
  mediagis/nominatim:4.2
'''

#### Try 4: pg_dump/pg_restore

https://nominatim.org/release-docs/latest/admin/Advanced-Installations/#moving-the-database-to-another-machine